In [1]:
# import SparkSession
from pyspark.sql import SparkSession

# Create SparkSession
spark = SparkSession.builder.appName("Data Analysis").getOrCreate()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1732380342473_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# Load data
api_df = spark.read.csv('s3://project-buck3/TRUSTED/api_db/cleaned_data.csv',inferSchema=True,header=True)
local_df = spark.read.csv('s3://project-buck3/TRUSTED/local_db/cleaned_data.csv',inferSchema=True,header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# Reconocimiento del dataframe
def print_df_columns(df, df_name):
    print(f'{df_name} dataframe columns:')
    for column in df.columns:
        print(f' - {column}')
    print()
    
print_df_columns(api_df, 'Covid api data')
print_df_columns(local_df, 'Covid local data')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Covid api data dataframe columns:
 - fecha_reporte
 - departamento_id
 - departamento_nombre
 - municipio_id
 - municipio_nombre
 - edad
 - sexo
 - recuperado
 - fecha_inicio_sintomas
 - fecha_diagnostico
 - fecha_recuperacion
 - fecha_muerte

Covid local data dataframe columns:
 - fecha_reporte
 - pruebas_realizadas
 - personas_vacunadas
 - ocupacion_uci
 - tasa_transmision
 - medidas_control

In [27]:
api_df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

973

In [28]:
local_df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1413

In [29]:
api_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- fecha_reporte: date (nullable = true)
 |-- departamento_id: integer (nullable = true)
 |-- departamento_nombre: string (nullable = true)
 |-- municipio_id: integer (nullable = true)
 |-- municipio_nombre: string (nullable = true)
 |-- edad: integer (nullable = true)
 |-- sexo: string (nullable = true)
 |-- recuperado: string (nullable = true)
 |-- fecha_inicio_sintomas: string (nullable = true)
 |-- fecha_diagnostico: date (nullable = true)
 |-- fecha_recuperacion: string (nullable = true)
 |-- fecha_muerte: string (nullable = true)

In [30]:
local_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- fecha_reporte: date (nullable = true)
 |-- pruebas_realizadas: integer (nullable = true)
 |-- personas_vacunadas: integer (nullable = true)
 |-- ocupacion_uci: double (nullable = true)
 |-- tasa_transmision: double (nullable = true)
 |-- medidas_control: string (nullable = true)

In [31]:
api_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+---------------+-------------------+------------+----------------+----+----+----------+---------------------+-----------------+------------------+------------+
|fecha_reporte|departamento_id|departamento_nombre|municipio_id|municipio_nombre|edad|sexo|recuperado|fecha_inicio_sintomas|fecha_diagnostico|fecha_recuperacion|fecha_muerte|
+-------------+---------------+-------------------+------------+----------------+----+----+----------+---------------------+-----------------+------------------+------------+
|   2020-07-18|             44|            GUAJIRA|       44001|        RIOHACHA|  38|   F|Recuperado|           2020-06-27|       2020-07-11|        2020-07-27|         N/A|
|   2021-01-07|             76|              VALLE|       76001|            CALI|  67|   M|Recuperado|           2020-12-27|       2021-01-05|        2021-02-26|         N/A|
|   2020-07-09|             11|             BOGOTA|       11001|          BOGOTA|  50|   F|Recuperado|           2020-06-29| 

In [32]:
local_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+------------------+------------------+-------------+----------------+--------------------+
|fecha_reporte|pruebas_realizadas|personas_vacunadas|ocupacion_uci|tasa_transmision|     medidas_control|
+-------------+------------------+------------------+-------------+----------------+--------------------+
|   2020-10-13|               458|                69|        96.69|            2.05|   Sin restricciones|
|   2021-06-18|               264|               130|        87.87|            0.93| Cierre de fronteras|
|   2021-10-13|               678|               751|        77.06|            1.97|   Sin restricciones|
|   2022-06-05|               886|               163|        71.11|            2.05| Cuarentena estricta|
|   2022-09-12|               566|               732|        55.93|            1.78|Restricción nocturna|
|   2023-09-24|               653|               859|        92.79|            1.51| Cierre de fronteras|
|   2023-10-20|               420|            

In [56]:
trusted_dir = "s3://project-buck3/REFINED/bussiness_analytics/dataframes"


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
from pyspark.sql.functions import col

# Total de casos por departamento y municipio
casos_por_ubicacion = api_df.groupBy("departamento_nombre", "municipio_nombre").count()
casos_por_ubicacion.orderBy(col("count").desc()).show()


casos_por_ubicacion.coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save(trusted_dir + "casos_por_ubicacion") 


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----------------+-----+
|departamento_nombre|municipio_nombre|count|
+-------------------+----------------+-----+
|             BOGOTA|          BOGOTA|  174|
|              VALLE|            CALI|  164|
|          ANTIOQUIA|        MEDELLIN|   77|
|          ANTIOQUIA|           BELLO|   63|
|          CARTAGENA|       CARTAGENA|   51|
|       BARRANQUILLA|    BARRANQUILLA|   51|
|          SANTANDER| BARRANCABERMEJA|   43|
|          ANTIOQUIA|          ITAGUI|   30|
|          SANTANDER|     BUCARAMANGA|   22|
|             TOLIMA|          IBAGUE|   22|
|          SANTANDER|   FLORIDABLANCA|   21|
|              SUCRE|       SINCELEJO|   20|
|               META|   VILLAVICENCIO|   17|
|          SANTANDER|     PIEDECUESTA|   12|
|     STA MARTA D.E.|     SANTA MARTA|   12|
|             CALDAS|       MANIZALES|   10|
|              CAUCA|         POPAYAN|    9|
|            CORDOBA|        MONTERIA|    9|
|          ANTIOQUIA|        RIONEGRO|    6|
|         

In [58]:
from pyspark.sql.functions import to_date

# Convertir fechas a formato adecuado
df = api_df.withColumn("fecha_reporte", to_date("fecha_reporte", "yyyy-MM-dd"))

# Casos reportados por día
casos_por_dia = df.groupBy("fecha_reporte").count()
casos_por_dia.orderBy(col("count").desc()).show()

casos_por_dia.coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save(trusted_dir + "casos_por_dia") 


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-----+
|fecha_reporte|count|
+-------------+-----+
|   2020-09-03|  180|
|   2020-07-18|  134|
|   2020-06-25|   96|
|   2021-01-07|   70|
|   2020-11-08|   61|
|   2021-01-14|   60|
|   2020-07-25|   54|
|   2020-08-05|   49|
|   2020-09-23|   45|
|   2020-12-24|   30|
|   2020-08-13|   30|
|   2020-10-23|   29|
|   2021-01-13|   24|
|   2020-12-08|   21|
|   2020-07-09|   18|
|   2020-09-13|   15|
|   2021-01-12|   13|
|   2020-10-03|   12|
|   2020-08-17|   11|
|   2020-08-25|   10|
+-------------+-----+
only showing top 20 rows

In [60]:
# Distribución de casos por edad
casos_por_edad = api_df.groupBy("edad").count()
casos_por_edad.orderBy(col("count").desc()).show()

casos_por_edad.coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save(trusted_dir + "casos_por_edad") 


# Distribución por sexo
casos_por_sexo = df.groupBy("sexo").count()
casos_por_sexo.show()

casos_por_sexo.coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save(trusted_dir + "casos_por_sexo") 


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+
|edad|count|
+----+-----+
|  33|   37|
|  32|   34|
|  30|   27|
|  35|   27|
|  25|   24|
|  26|   23|
|  40|   23|
|  52|   23|
|  22|   22|
|  23|   22|
|  31|   22|
|  42|   22|
|  38|   21|
|  24|   21|
|  29|   21|
|  34|   21|
|  41|   20|
|  37|   19|
|  36|   19|
|  39|   19|
+----+-----+
only showing top 20 rows

+----+-----+
|sexo|count|
+----+-----+
|   M|  540|
|   F|  433|
+----+-----+

In [61]:
# Evaluar el porcentaje de recuperados por sexo
recuperacion_por_sexo = api_df.groupBy("sexo") \
    .agg(
        count(when(col("recuperado") == "Recuperado", True)).alias("casos_recuperados"),
        count("*").alias("total_casos")
    ) \
    .withColumn("porcentaje_recuperacion", (col("casos_recuperados") / col("total_casos")) * 100)
recuperacion_por_sexo.show()

recuperacion_por_sexo.coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save(trusted_dir + "recuperacion_por_sexo") 


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------------+-----------+-----------------------+
|sexo|casos_recuperados|total_casos|porcentaje_recuperacion|
+----+-----------------+-----------+-----------------------+
|   M|              518|        540|      95.92592592592592|
|   F|              415|        433|      95.84295612009238|
+----+-----------------+-----------+-----------------------+

In [62]:
from pyspark.sql.functions import when

# Filtrar casos recuperados
recuperados = df.filter(col("recuperado") == "Recuperado")

# Contar casos por departamento
tasa_recuperacion = recuperados.groupBy("departamento_nombre").count()
tasa_recuperacion.orderBy(col("count").desc()).show()

tasa_recuperacion.coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save(trusted_dir + "tasa_recuperacion_departamento") 


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-----+
|departamento_nombre|count|
+-------------------+-----+
|          ANTIOQUIA|  217|
|             BOGOTA|  166|
|              VALLE|  148|
|          SANTANDER|  101|
|          CARTAGENA|   51|
|       BARRANQUILLA|   51|
|              SUCRE|   25|
|             TOLIMA|   24|
|               META|   16|
|       CUNDINAMARCA|   15|
|             CALDAS|   12|
|          RISARALDA|   12|
|     STA MARTA D.E.|   12|
|            GUAJIRA|   12|
|              CAUCA|   10|
|          ATLANTICO|   10|
|            CORDOBA|   10|
|            BOLIVAR|    8|
|    NORTE SANTANDER|    7|
|              HUILA|    6|
+-------------------+-----+
only showing top 20 rows

In [63]:
# Listar los municipios con mayor porcentae de muertes respecto al total de casos 
mortalidad_municipios = api_df.groupBy("municipio_nombre") \
    .agg(
        count(when(col("fecha_muerte") != "N/A", True)).alias("total_muertes"),
        count("*").alias("total_casos")
    ) \
    .withColumn("porcentaje_mortalidad", (col("total_muertes") / col("total_casos")) * 100) \
    .orderBy("porcentaje_mortalidad", ascending=False)
mortalidad_municipios.show()

mortalidad_municipios.coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save(trusted_dir + "mortalidad_municipios") 


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-------------+-----------+---------------------+
|municipio_nombre|total_muertes|total_casos|porcentaje_mortalidad|
+----------------+-------------+-----------+---------------------+
|           FUNZA|            1|          1|                100.0|
|        CAUCASIA|            1|          1|                100.0|
|          OBANDO|            1|          1|                100.0|
|         ARMENIA|            1|          2|                 50.0|
|          SOACHA|            1|          6|   16.666666666666664|
|            CALI|           20|        164|   12.195121951219512|
|     PIEDECUESTA|            1|         12|    8.333333333333332|
|   VILLAVICENCIO|            1|         17|     5.88235294117647|
|          BOGOTA|            8|        174|    4.597701149425287|
|     BUCARAMANGA|            1|         22|    4.545454545454546|
|    BARRANQUILLA|            2|         51|   3.9215686274509802|
|        MEDELLIN|            2|         77|   2.5974025974025

In [64]:
from pyspark.sql.functions import avg,datediff

# Calcular días entre síntomas y diagnóstico
df = df.withColumn("dias_sintomas_diagnostico", datediff("fecha_diagnostico", "fecha_inicio_sintomas"))

# Calcular días entre diagnóstico y recuperación
df = df.withColumn("dias_diagnostico_recuperacion", datediff("fecha_recuperacion", "fecha_diagnostico"))

# Promedio de días
tiempos_promedio = df.select(
    avg("dias_sintomas_diagnostico").alias("promedio_dias_sintomas_diagnostico"),
    avg("dias_diagnostico_recuperacion").alias("promedio_dias_diagnostico_recuperacion")
)
tiempos_promedio.show()

tiempos_promedio.coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save(trusted_dir + "tiempos_promedio") 


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------------------+--------------------------------------+
|promedio_dias_sintomas_diagnostico|promedio_dias_diagnostico_recuperacion|
+----------------------------------+--------------------------------------+
|                11.247658688865766|                     23.19506966773848|
+----------------------------------+--------------------------------------+

In [65]:
# Promedio y mediana por municipio entre la fecha de diagnóstico y de recuperación

from pyspark.sql.window import Window
from pyspark.sql.functions import percentile_approx

# Calcular días de recuperación
api_df = api_df.withColumn("dias_recuperacion", 
    datediff(col("fecha_recuperacion"), col("fecha_inicio_sintomas")))

dias_recuperacion_estadisticas = api_df.groupBy("municipio_nombre") \
    .agg(
        avg("dias_recuperacion").alias("promedio_dias_recuperacion"),
        percentile_approx("dias_recuperacion", 0.5).alias("mediana_dias_recuperacion")
    ) \
    .orderBy("promedio_dias_recuperacion")
dias_recuperacion_estadisticas.show()

dias_recuperacion_estadisticas.coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save(trusted_dir + "estadisticas_recuperacion_municipios") 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------------------+-------------------------+
|   municipio_nombre|promedio_dias_recuperacion|mediana_dias_recuperacion|
+-------------------+--------------------------+-------------------------+
|              FUNZA|                      NULL|                     NULL|
|           CAUCASIA|                      NULL|                     NULL|
|             OBANDO|                      NULL|                     NULL|
|            CAIMITO|                      13.0|                       13|
|            CARTAGO|                      15.0|                       15|
|            LA CEJA|                      15.0|                       15|
|            DUITAMA|                      15.0|                       15|
|          ZIPAQUIRA|                      15.0|                       15|
|           LA UNION|                      15.0|                       15|
|SANTA ROSA DE CABAL|                      15.0|                       15|
|        PUERTO RICO|    

In [45]:
from pyspark.sql.functions import col, avg, count, when, lit

# Unión de los DataFrames en base a fecha_reporte
combined_df = api_df.join(local_df, on="fecha_reporte", how="inner")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
# Consulta 2: Relación entre tasa de transmisión y personas vacunadas
consulta_2 = combined_df.groupBy("tasa_transmision") \
    .agg(
        avg("personas_vacunadas").alias("promedio_vacunados"),
        count("*").alias("total_registros")
    ).orderBy(col("tasa_transmision"))
consulta_2.show()

consulta_2.coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save(trusted_dir + "transmision_vacunacion") 


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+------------------+---------------+
|tasa_transmision|promedio_vacunados|total_registros|
+----------------+------------------+---------------+
|            0.82|             910.0|              4|
|            0.86|             422.0|             21|
|            0.91|             802.0|             96|
|            0.95|             997.0|             70|
|            0.97|             584.0|             18|
|            0.98|             212.0|             10|
|            0.99|             157.0|             54|
|            1.15|             648.0|             13|
|             1.2|             887.0|             24|
|            1.27|             643.0|            134|
|             1.3|             911.0|             11|
|            1.39|             779.0|              1|
|            1.51|             943.0|             30|
|            1.63|             521.0|              1|
|            1.69| 201.0909090909091|             33|
|            1.79|          

In [67]:
# Consulta 3: Comparar casos recuperados con medidas de control
consulta_3 = combined_df.groupBy("medidas_control") \
    .agg(
        count(when(col("recuperado") == "Recuperado", True)).alias("total_recuperados"),
        count("*").alias("total_casos")
    ).withColumn(
        "porcentaje_recuperados",
        (col("total_recuperados") / col("total_casos")) * 100
    )
consulta_3.show()

consulta_3.coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save(trusted_dir + "medidas_control") 


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------------+-----------+----------------------+
|     medidas_control|total_recuperados|total_casos|porcentaje_recuperados|
+--------------------+-----------------+-----------+----------------------+
|   Sin restricciones|              130|        146|     89.04109589041096|
| Cuarentena estricta|              343|        353|     97.16713881019831|
|Restricción nocturna|              216|        220|     98.18181818181819|
| Cierre de fronteras|              244|        254|     96.06299212598425|
+--------------------+-----------------+-----------+----------------------+

In [68]:
# Consulta 4: Relación entre edad promedio y ocupación UCI
consulta_4 = combined_df.groupBy("fecha_reporte") \
    .agg(
        avg("edad").alias("edad_promedio"),
        avg("ocupacion_uci").alias("ocupacion_uci_promedio")
    ).orderBy(col("fecha_reporte"))
consulta_4.show()

consulta_4.coalesce(1).write.format("csv").option("header", "true").mode("overwrite").save(trusted_dir + "edad_ocupacion_UCI") 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+------------------+----------------------+
|fecha_reporte|     edad_promedio|ocupacion_uci_promedio|
+-------------+------------------+----------------------+
|   2020-05-23|              45.0|                 92.47|
|   2020-06-25|38.572916666666664|     75.84000000000009|
|   2020-07-09| 43.55555555555556|                 75.77|
|   2020-07-18| 37.12686567164179|     80.36000000000003|
|   2020-07-25| 43.05555555555556|     66.34999999999994|
|   2020-08-05|39.775510204081634|     54.55000000000001|
|   2020-08-09|             27.75|                 77.54|
|   2020-08-13| 44.63333333333333|     88.78999999999998|
|   2020-08-17| 47.54545454545455|                 66.52|
|   2020-08-25|              48.4|     72.47000000000001|
|   2020-09-03|40.766666666666666|    54.579999999999934|
|   2020-09-13| 34.06666666666667|     96.32999999999998|
|   2020-09-23| 45.15555555555556|                 66.39|
|   2020-09-28|              31.0|                 76.51|
|   2020-10-03